# Project 2 report (Group 15)
GitHub repository: https://github.com/pypr23/project-2-group-15

# Introduction

The public health system is increasingly worried in the post-pandemic age. The public health system in the United Kingdom (UK) is primarily centered on the National Health Service (NHS), which offers free health care to all UK residents based on need rather than ability to pay. One component of NHS is the GP service, which refers to primary health care provided by general practitioners (GPs) in the United Kingdom. In recent years, GP services have always been criticized, such as long wait times and poor quality. Patients at a Lanarkshire practice, for example, claim they have been unable to schedule appointments due to unmanageably long waiting lists and overworked physicians, putting their lives in danger (https://www.heraldscotland.com/news/23917188.scottish-gp-group-looked-serious-complaints/). GP services can be roughly thought of as a demand and supply issue. Infection concerns are represented by demand, and GP services are represented by supply.Therefore, in order to determine the true situation of UK public healthcare, an investigation into infection situation and GP supply will be conducted, employing Python to present and assess data. There is also would like to give some advice. To begin with, please run the following code in order to make sure every code is runable in coming sections.

## Package Going to Be Used

In [ ]:
import importlib
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import seaborn as sns
from matplotlib import pyplot as plt
import datetime as dt
import ipywidgets as widgets
from IPython.display import display
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import geopandas
import mapclassify
from plotly.subplots import make_subplots
import plotly.express as px
import branca as bc
import folium
from folium import plugins


### General Function
import Functions.General as general

### Infection Issuses Investigation Functions
import Functions.prevalence as pr

### GP Analysis Functions
import Functions.GP as gp_func

### Prescription Functions
import Functions.prescription as rx

## Data Going to be Used

### Prevalence Data

In [ ]:
redownload = True
#reloading used library for this subsection
importlib.reload(general);

In [ ]:
prevalence_data = general.get_data("diseaseprevalence_scotland_total.csv", 'https://publichealthscotland.scot/media/21148/diseaseprevalence_scotland_total.xlsx', redownload, sheet_name="DP_Scotland_total")
prevalence_data_female = general.get_data('diseaseprevalence_scotland_f.csv','https://publichealthscotland.scot/media/21146/diseaseprevalence_scotland_f.xlsx', redownload, sheet_name="DP_Scotland_F")
prevalence_data_male = general.get_data('diseaseprevalence_scotland_m.csv','https://publichealthscotland.scot/media/21147/diseaseprevalence_scotland_m.xlsx', redownload, sheet_name="DP_Scotland_M")

In [ ]:
#General Data Cleaning
prevalence_data_all_age = prevalence_data[prevalence_data['Age']=="All"].copy()
prevalence_data_all_age = prevalence_data.groupby(['Year']).sum(numeric_only = True)

#Calculating Infection Population, this means that it's regardless of demographics or illness kind
prevalence_data_all_age_total = pd.DataFrame()
prevalence_data_all_age_count = prevalence_data_all_age.iloc[0:,1:20].copy()
prevalence_data_all_age_total["Total (Count)"] = prevalence_data_all_age_count.sum(axis=1)

Data Resource: https://www.scotlandscensus.gov.uk/documents/scotland-s-census-2022-rounded-population-estimates-data/

In [ ]:
#Population Data
population = general.get_data('scotland-s-census-2022-first-results-rounded-population-estimates-data.csv','Data/Population/scotland-s-census-2022-first-results-rounded-population-estimates-data.xlsx', redownload, sheet_name="Figure 1", skiprows=[0,1,2])
population = population[17:].set_index('Year')

In [ ]:
#Prevalence Data Divided by Board Division
prevalence_data_board = general.get_data('diseaseprevalence_board_total.csv','https://publichealthscotland.scot/media/21136/diseaseprevalence_board_total.xlsx', redownload, sheet_name = "DP_Board_total")

### GP Data

In [ ]:
redownload = False
#reloading used library for this subsection
importlib.reload(general);

In [ ]:
GP_workforce = general.get_data('table1_number_of_gps_in_scotland_by_age_designation_sex_2012_2022.csv','https://publichealthscotland.scot/media/16746/table1_number_of_gps_in_scotland_by_age_designation_sex_2012_2022.xlsx', redownload, sheet_name="Sex", header=5, skipfooter=3)
GP_workforce = GP_workforce.rename(columns={"Unnamed: 0":"Gender"})

In [ ]:
GP_practice_list = general.get_data('practice_contactdetails_oct2023.csv','https://publichealthscotland.scot/media/23639/practice_contactdetails_oct2023.xlsx', redownload, sheet_name="Practice_ContactDetails", skiprows=5)
GP_practice_list["NHS Board Name"] = GP_practice_list["NHS Board Name"].str.replace("&","and")

In [ ]:
#GP demographics data
GP_demographics = general.get_data('demographics_2023_q1.csv','https://publichealthscotland.scot/media/22255/demographics_2023_q1.xlsx', redownload, sheet_name = "Age-Gender")
GP_demographics.rename(columns={"Board":"HBName"}, inplace=True)
GP_demographics["Year"] = GP_demographics["Quarter"].str.slice(stop=4)
GP_demographics["HBName"] = GP_demographics["HBName"].str.slice(start=3)
GP_demographics["HBName"] = GP_demographics["HBName"].str.replace("&","and")

In [ ]:
#GP workforce data divided by Board
GP_board = general.get_data('table2_number_of_gps_nhsboard_designation_sex_2012_2022.csv','https://publichealthscotland.scot/media/16747/table2_number_of_gps_nhsboard_designation_sex_2012_2022.xlsx', redownload, sheet_name="Data", header=2)
GP_board.drop("Lookup", axis="columns", inplace=True)
GP_board["NHS Board"] = GP_board["NHS Board"].str.replace("&","and")

Data Resource: https://www.nrscotland.gov.uk/statistics-and-data/geography/our-products/scottish-postcode-directory/2023-2

In [ ]:
#There will be a DtypeWarning but that problematic columns will not be used
Small_user = general.get_data_zip('SmallUser.csv','https://www.nrscotland.gov.uk/files//geography/2023-2/SPD_PostcodeIndex_Cut_2023_2_CSV.zip', redownload)
Small_user = Small_user[Small_user['DateOfDeletion'].isnull()].copy()

Large_user = general.get_data_zip('LargeUser.csv','https://www.nrscotland.gov.uk/files//geography/2023-2/SPD_PostcodeIndex_Cut_2023_2_CSV.zip', redownload)
Large_user = Large_user[Large_user['DateOfDeletion'].isnull()].copy()


### Prescription Data

Researchers can import the necessary data files in this section of the code.

In [ ]:
redownload = False
#reloading used library for this subsection
importlib.reload(general);
importlib.reload(rx);

Resources: https://applications.nhsbsa.nhs.uk/infosystems/data/showDataSelector.do?reportId=126

In [ ]:
#BNF Item Code Reference 2023
prescription_code = general.get_data("20231101_1698828903284_BNF_Code_Information.csv", "Data/BNF Code/20231101_1698828903284_BNF_Code_Information.csv", redownload)
prescription_code['BNF Presentation'] = prescription_code['BNF Presentation'].str.lower()

In [ ]:
#Prescription data in 07/2023
prescription_data = general.get_data("pitc202307.csv", "https://www.opendata.nhs.scot/dataset/84393984-14e9-4b0d-a797-b288db64d088/resource/7bb45ee6-6f1c-45f4-933b-958bdbe0ca4f/download/pitc202307.csv", redownload)
prescription_data = rx.unify(prescription_data)
prescription_data['BNF Presentation Code'] = prescription_data['BNF Presentation Code'].astype(str)

In [ ]:
#Prescription data over three years (202007 - 202306)
base_dir = r"Data"
dates = [f'{year}{month:02d}' for year in range(2020, 2023+1) for month in range(1, 13)]
dates = [date for date in dates if not (date.startswith("2020") and int(date[-2:]) < 7) and not (date.startswith("2023") and int(date[-2:]) > 6)]

#Get the file link from the website
url = 'https://www.opendata.nhs.scot/dataset/prescriptions-in-the-community'
web = requests.get(url)                        
soup = BeautifulSoup(web.text, "html.parser")  
result = soup.findAll('a', attrs={'class':'resource-url-analytics'})
links = []
for h in result:
    link = h.get('href')
    if any(date in link for date in dates):
        links.append(link)
        
#Import all datasets, including the columns 'BNFItemCode' and 'PaidDateMonth'. 
#Rename the column 'BNFItemCode' to 'BNFCode'.   
prescription_data_3yrs = pd.concat(
    [general.get_data(f"pitc{dates[n]}.csv", links[n], redownload, usecols=['BNFItemCode', 'PaidDateMonth']).rename(columns={'BNFItemCode': 'BNFCode'}) 
     for n in range(len(links))]
)

#Sort the data by the 'PaidDateMonth' column in date order.
prescription_data_3yrs = prescription_data_3yrs.sort_values(by='PaidDateMonth')

### Shape File for Plotting Maps

In [ ]:
#Loading the shape file
path = "Data/shapefile/SG_NHS_HealthBoards_2019.shp"
shape_df = geopandas.read_file(path)
shape_df = shape_df.set_index('HBName')

# Analysis

As the **Board division** is the **largest division of NHS Services**, to obtain an **overview geographic distribution** for each data, the Board division will be used in this report.

### Prevalence Study (Demand)

In [ ]:
#reloading used library for this subsection
importlib.reload(pr);

#### Trend of Infected Population

In [ ]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=('Infectious Population from 2018 to 2023','Population Trend from 1981 to 2022'))
#Scatter Plot of Infectious Population
fig.add_trace(go.Scatter(x=prevalence_data_all_age_total.index, y=prevalence_data_all_age_total["Total (Count)"],name='Infectious Population'),row=1, col=1)

#Scatter Plot of Population
fig.add_trace(go.Scatter(x=population.index, y=population["All persons\n[Note 2]"], name='Population'), row=1, col=2)

#Setting up the axes
fig.update_yaxes(title_text="Population", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_traces(marker=dict(color='#638583'), row=1, col=1)
fig.update_traces(marker=dict(color='#ECC558'), row=1, col=2)

fig.update_layout(height=600, width=1300, title_text="Comparision between Infectious Population and Population", xaxis2=dict(tickvals=population.index))
fig.show()

It is more fair to make the visual comparison with the y-axis limit set to the same.

Observations:
1.   Surprisingly, the **diseased population fell from 2020 to 2021**. It is speculated that this is due to the UK government implementing lockdown from 2020 to 2021.
2.   Besides, According to the graph above, both the **population** and the **diseased population** are **growing**. From 2018 to 2023, **infectious population increases around 8.9%**. To elaborate, the number of infections should be proportional to the population. The **[National Records of Scotland](https://www.nrscotland.gov.uk/statistics-and-data/statistics/statistics-by-theme/population/population-projections/population-projections-scotland/2020-based)** state that higher migration estimates will cause the **population to grow until roughly mid-2033**, when it is expected to peak at 5.53 million, increasing the number of affected people. It is expected to fall by 0.6% to 5.49 million by 2045, but if the 2018 baseline is used, the projection could ascend. Simply speaking, the **demand of GP services will be increased**. To be more certain, the infectious population will also be predicted.

#### Infectious Population Prediction

Common Prediction Methodologies:

1.   **Regression models:** It is assumed that the residual follows a **normal distribution**, which has a high probability of being true when the amount of data is substantial. However, because there is insufficient data, Regression is **NOT** possible.
2.   **Time Series Analysis (e.g., the ARIMA model):** The charts above have previously demonstrated that the data is not steady, hence **invalidating the ARIMA model's stationary assumption**. Again, because the amount of data is small, alternative time series analyses, such as the GARCH model or even taking a difference to do the ARIMA model, will not function well. Time Series Analysis is thus **NOT** feasible.

Eventually, there is going to attempt to employ **Monte-Carlo Forecasting**.

Remarks:

1. `Prediction Year`: means how many years would like to predict (e.g. 10 means 10 years from 2023). This value must be **larger than 1**. Since prediction error will increase according to the increasing of year, the **maximum** of this value should be **100**.
2. `Trials`: means how many possibilities would like to see (e.g. 50 means 50 lines/possibliities). This value must be **larger than 0**. Since increasing the number of trials will increase the computational time and it is not necessary for too many possibilities, the **maximum** of this value should be **500**

*If you are unable to run the code, please refer to the two demonstration videos in the 'Interactive Plots Demonstration' folder.*

In [ ]:
semi_inter = True #Change it to be False to display a full auto simulation plot
if semi_inter:    
    pr.mc_sim_semi_interactive(df = prevalence_data_all_age_total)
else:
    pr.mc_sim_interactive(df = prevalence_data_all_age_total)

In [ ]:
sns.set()
pr.mc_sim_non_interactive(prevalence_data_all_age_total)

The infectious population for the **current year (2023) is represented by the black horizontal line**. Each line represents a distinct possibility. While the infected population may drop in the coming years, the **infectious population will increase overall**, according to the line chart below. Therefore, before resolving this issue, the existing situation should be reviewed.

#### Current Status of Infection Statistics

##### Common Diseases

In [ ]:
plot_df = prevalence_data_all_age.copy()
type = ['Patients Counting','Rate']
for t in type:
  pr.bar_visual(t,plot_df)

As can be seen from the above plots, the average number of persons affected by **hypertension *(1057758)***, **depression *(6189370)***, and **asthma *(5184187)*** between 2018 and 2023 is a clear indication of the prevalent illnesses.  Their relative prevalence rates are *338.6*, *137.1*, and *123.8*. In the United Kingdom, for example, an estimated 339 individuals out of every 100,000 have hypertension.

In [ ]:
plot_df = prevalence_data_all_age.copy()
pr.bar_visual("Change",plot_df)

Surprisingly, the **top three diseases that are increasing** in prevalence are **eating disorders**, **COPD**, and **stroke and TIA**.

##### Prevalence vs Demographics Satitistics (i.e. age group and gender)

According to the dataset's definition of prevalence rate, **prevalence is how widespread an illness is in a population**. **Prevalence rate** will therefore be used in this section to determine the distribution within a given population group.

In [ ]:
top_num = 3 #Change this number if you want to see more diseases distribution
prevalence_data_age = prevalence_data[prevalence_data['Age']!="All"].copy()
prevalence_data_age = prevalence_data_age.groupby(['Age']).mean(numeric_only = True)
prevalence_data_age_count = prevalence_data_age.iloc[:,40:].copy()
top = [t.replace("Rate_", "" ) for t in pr.sorting(prevalence_data_age_count.mean(numeric_only=True),top_num)]
print(f"Distribution plot for showing {top_num} most common diseases distribution on averge from 2018 to 2023 by gender")
pr.dist_visual(top,prevalence_data_male, prevalence_data_female)

The graph above illustrates the spread of three most prevalent illnesses in the United Kingdom by **gender** and **age group**.

1. It clearly demonstrates that age is connected to the type of disease.  For instance,  **when a person reach a certain age, he/she is more prone to get hypertension**.
2. It also states unequivocally that gender is related to the infection. **Females** have an **increased risk of contracting these three common diseases**. **Female *depression prevalence* is substantially higher than male depression prevalence.** In other words, greater attention should be paid to female mental health.

##### Prevalence vs Location

In [ ]:
#Preparing Data for Analysis
prevalence_data_board_all_age_board = prevalence_data_board[prevalence_data_board['Age'] == "All"].copy()
prevalence_data_board_all_age_board = prevalence_data_board_all_age_board.groupby(['GPPractice/Area', 'Year']).sum(numeric_only = True)
prevalence_data_board_all_age_board = prevalence_data_board_all_age_board.groupby([ 'GPPractice/Area']).mean(numeric_only = True)
index = [id.replace("NHS ", "") for id in prevalence_data_board_all_age_board.index]
prevalence_data_board_all_age_board['HBName'] = index
prevalence_data_board_all_age_board = prevalence_data_board_all_age_board.set_index('HBName')
prevalence_data_board_all_age_board['HBName'] = index
prevalence_data_board_all_age_board['HBName'] = prevalence_data_board_all_age_board['HBName'].str.replace("&","and")
df_plot_map = pd.melt(prevalence_data_board_all_age_board.iloc[:,39:], id_vars="HBName", var_name="Diseases", value_name="Prevalence Rates")
df_plot_map["Diseases"]= df_plot_map["Diseases"].str.replace("Rate_","")
df_plot_map['Info'] = [f'{df_plot_map.iloc[i,0]} : {round(df_plot_map.iloc[i,2],2)}' for i in  range(df_plot_map.shape[0])]
df_plot_map = df_plot_map.set_index('HBName')

In [ ]:
top_num = 3 #Change this number if you want to see more diseases distribution
prevalence_data_age = prevalence_data[prevalence_data['Age']!="All"].copy()
prevalence_data_age = prevalence_data_age.groupby(['Age']).mean(numeric_only = True)
prevalence_data_age_count = prevalence_data_age.iloc[:,40:].copy()
top = [t.replace('Rate_','') for t in pr.sorting(prevalence_data_age_count.mean(numeric_only=True), top_num)]
print(f"Map for showing {top_num} most common diseases distribution on averge from 2018 to 2023")
pr.map_visual(df_plot_map, shape_df, top)

The graph above depicts the geographic distribution of three most prevalent diseases in the United Kingdom.

1. It (the most left graph) appears that the **prevalence of hypertension** will be **higher in more northern NHS Board areas**.
2. For **others top spread diseases**, the **distribution appears to be uniform**.
3. There should be awared that the **prevalence of three diseases are also very high in  Shetland**

In [ ]:
top_num = 3 #Change this number if you want to see more diseases distribution
prevalence_data_age = prevalence_data[prevalence_data['Age']!="All"].copy()
prevalence_data_age = prevalence_data_age.groupby(['Age']).mean(numeric_only = True)
prevalence_data_age_count = prevalence_data_age.iloc[:,40:].copy()
top = [t.replace('Rate_','') for t in pr.sorting(prevalence_data_age_count.mean(numeric_only=True),top_num)]
disease = 0
index = []
while True:
    print("Which interactive map would like to display?")
    print("Common Diseases:")
    for t in range(len(top)):
        print(f"{t}. {top[t]}")
        index.append(str(t))
    disease = input(f"Please enter {[t for t in range(len(top))]}")
    if disease not in index:
        print(f"Please Enter {[t for t in range(len(top))]}, {disease} is not a appropriate input!")
    else:
        disease = int(disease)
        break;
print(f"Generating Interactive Map for {top[disease]}...")
df_plot = df_plot_map[df_plot_map['Diseases']==top[disease]].copy()
geo_df_prevalence = shape_df.join(df_plot)
geo_df_prevalence['Board'] = geo_df_prevalence.index
geo_df_prevalence['Info'] = [f'{geo_df_prevalence.iloc[i,-1]} : {round(geo_df_prevalence.iloc[i,5],2)}' for i in  range(geo_df_prevalence.shape[0])]
geo_df_prevalence.explore(column = "Prevalence Rates", tooltip = "Info", legend=True, cmap='YlOrRd', missing_kwds={'color': 'white', 'edgecolor': 'red','hatch': '///','label': 'Missing values'})

### GP Practice Analysis (Supply)

In [ ]:
#reloading used library for this subsection
importlib.reload(gp_func); 

#### Trend of GP Workforce

In [ ]:
GP_workforce_plot = pd.melt(GP_workforce, id_vars="Gender", var_name="Year", value_name="Number of Workforce")
gp_func.line_chart(GP_workforce_plot)


From **2018 to 2022**, the **overall GP workforce increased by about 4.5%**, whereas the **infectious population increased by about 7%**. To further explain, the **growing GP workforce cannot keep up with the growing infectious population**. Besides, the number of **men** in the GP workforce **exceeds** that of **female**, and it is **increasing**. The **chasm between the two is growing bigger**. In other words, ***female's* labor-force participation is dropping while *men's* labor-force participation is increasing**.

#### GP Location Distribution

In [ ]:
GP_practice_list_count = pd.DataFrame({"Number of GP":GP_practice_list.groupby("NHS Board Name").size()})
GP_practice_list_count["HBName"] = GP_practice_list_count.index
px.bar(GP_practice_list_count,x="HBName",y="Number of GP", color="HBName", height=800, title="Number of GP by Board Region in 2023")

In [ ]:
GP_demographics_patients = GP_demographics.groupby(["Year","HBName"]).sum(numeric_only = True)
GP_demographics_patients = GP_demographics_patients.reset_index(level=[0,1])
px.bar(GP_demographics_patients,x="HBName",y="Total Patients", color="Year", barmode="group", height=800, title="Number of Patients by Board Region from 2018 to 2023")

In [ ]:
#Cleaning workforce dataset for plotting map
workforce_map = GP_board.groupby("NHS Board").sum(numeric_only = True).iloc[:,6:]
workforce_map = pd.DataFrame(workforce_map.mean(axis=1))
workforce_map.rename(columns={0:"Number of Workforce"}, inplace=True)

#Cleaning "number of patients" dataset for plotting map
GP_demographics_patients_map = GP_demographics_patients[GP_demographics_patients["Year"]!="2023"]
GP_demographics_patients_map= GP_demographics_patients_map.set_index("HBName")
GP_demographics_patients_map = GP_demographics_patients_map.iloc[:,:3]
GP_demographics_patients_map.drop("Practice", axis="columns", inplace=True)
GP_demographics_patients_map["HBName"] = GP_demographics_patients_map.index
GP_demographics_patients_map = pd.DataFrame(GP_demographics_patients_map.pivot(index="HBName", columns='Year').mean(axis=1))
GP_demographics_patients_map.rename(columns={0:"Number of Patients"}, inplace=True)
GP_demographics_patients_map = GP_demographics_patients_map.sort_index()

geo_df_GP = shape_df.join(workforce_map).join(GP_practice_list_count.drop(columns="HBName"))
geo_df_GP.sort_index()
geo_df_GP["Number of Patients"] = GP_demographics_patients_map["Number of Patients"].values

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(25,8))
geo_df_GP.plot('Number of Workforce', legend=True, cmap='BuPu', legend_kwds={"label":"Number of Workforce"}, ax=axes[0]).set(title=f"Location Distribution of Amount of Patients");
geo_df_GP.plot('Number of GP', legend=True, cmap='BuPu', legend_kwds={"label":"Number of GP"}, ax=axes[1]).set(title=f"Location Distribution of Number of GP");
geo_df_GP.plot('Number of Patients', legend=True, cmap='BuPu', legend_kwds={"label":"Number of Patients"}, ax=axes[2]).set(title=f"Location Distribution of Amount of Patients");
plt.suptitle("Location Distribution of 5 Years Average Values (from 2018 to 2022)");

1. The bar chart plainly shows that **Greater Glasgow and Clyde** has the **greatest number of GPs**, **patients** and **workforce**.
2. The map shows that the **number of GPs** in **Shetland** is very **low**, yet the prevalence rate of the three prevalent diseases listed in the preceding section is relatively high in that region. Furthermore, the **number of patients** is **NOT as low as** in **other places** such as Highland.

In [ ]:
All_user = Large_user.merge(Small_user,how='outer')
ALL = All_user.merge(GP_practice_list, on='Postcode')

practice_location = ALL[["Practice Name","Latitude","Longitude"]].copy()
# Create point geometries
geometry = geopandas.points_from_xy(practice_location.Longitude, practice_location.Latitude)
geo_df_heat_GP = geopandas.GeoDataFrame(practice_location, geometry=geometry)

map = folium.Map(location=[57.490669, -4.202646], tiles="OpenStreetMap", zoom_start=6.5)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in geo_df_heat_GP.geometry]
plugins.HeatMap(heat_data).add_to(map)
#Heatmap CANNOT display in Jupyter notebook
map.save("HeatMap/heatmap.html")
#map #for showing the map
#map.show_in_browser() #for showing the map directly in browser
print("The heatmap is succedssfully generated. You can find and check the map in folder called 'HeatMap'.")

According to the heatmap, **GP practice** is obviously **concentrated in the south of Scotland**, particularly in **Edinburgh** and **Glasgow**, Scotland's two major cities.

### Prescription Research (Resources)

#### Trend of Medication

Due to the limited resources of GPs, we have decided to redistribute medical resources to improve the medical service in the future.   To achieve this, we plan to identify the categories of medicine with the highest and lowest demand over a past period.   Based on this data, we will reduce the stock of medicines with lower demand and increase the supply of those with higher demand.   First, we need to import the database of prescriptions from the past period

##### Most In-demand Drugs

In [ ]:
#reloading used library for this subsection
importlib.reload(rx);

Below is an interactive plot where you type one month (YYYYMM) that you are interested, then enter the range of the top sales drugs you want to have a look.

In [ ]:
#Date input
#Get the minimum and maximum dates from the 'PaidDateMonth' column in the 'prescription_data_3yrs' dataframe.
min_date = prescription_data_3yrs['PaidDateMonth'].min()
max_date = prescription_data_3yrs['PaidDateMonth'].max()

#Ask the user to enter a start month in the format YYYYMM.
start_month = input(f"Enter the start month (from {min_date} to {max_date}) in format YYYYMM: ")

#Check if the entered start month is within the range of dates in the dataset.
if not (min_date <= int(start_month) <= max_date):
    # If not, print an error message with the valid date range.
    print(f"Error: The input range is beyond the date range. You should input a date from {min_date} to {max_date}")
else:

    # Process data to obtain section_counts for the entered start month.
    # Determine the range of data that user want to obtain
    # Filter the data for entries on and after the specified start month.
    filtered_data = prescription_data_3yrs[prescription_data_3yrs['PaidDateMonth'] >= int(start_month)]

    # Merge the filtered data with the BNF reference data.
    # We were given a dataset that recorded the use of different classes of drugs over a specified period of time. 
    # We grouped different drugs into categories according to their function and recorded their appearance of different categories.
    merged_data = pd.merge(filtered_data[['BNFCode']],
                           prescription_code[['BNF Presentation Code', 'BNF Section']],
                           left_on='BNFCode',
                           right_on='BNF Presentation Code',
                           how='left').drop(columns=['BNFCode', 'BNF Presentation Code'])

    # Count the occurrences of each BNF section (Therapeutic effect).
    section_counts = merged_data['BNF Section'].value_counts()

    # Get the total number of unique BNF sections (Therapeutic effect).
    max_sections = section_counts.count()

    # Ask the user to enter the number of top n popular medicine to be displayed, up to the maximum number of BNF sections in the user selected database.
    top_n_input = input(f"Enter the number of type of medicine to display (up to {max_sections}): ")
    top_n = int(top_n_input)

    # Check if the entered number is within the valid range.
    if not (0 < top_n <= max_sections):
        # If not, print an error message with the valid range.
        print(f"Error: The input range exceeds the maximum number of drug categories. Please input a number between 0 and {max_sections}")
    else:
        # If the inputs are valid, call the draw_charts function to display the charts.
        rx.draw_charts(top_n, start_month, prescription_data_3yrs, prescription_code, filtered_data, section_counts)

Based on the data from our pie and bar charts, medications such as **analgesics** and **drugs used in diabetes** have had the highest demand over the past three years, occupying a significant portion of the pie chart. Healthcare departments can increase the supply of these types of medications in the future to meet the potential future demand of the public.

##### The Least Demanded Medicines

Below is another interactive plot where you simply enter the range of the top sales drugs you want to have a look, then it will process a horizontal bar chart displaying the least demanded medicine categories in the same timeperiod of the top ones.

In [ ]:
#Ask the user to enter the number of least demand medicine categories to display, up to the maximum number of BNF sections
least_m_input = input(f"Enter the number of least demand medicine types to display (up to {max_sections}): ")

#Convert the input to integer and check if it's within the valid range
least_m = int(least_m_input)
if 0 < least_m <= max_sections:
    # Get the least demanding m drugs
    least_demand_sections = section_counts.nsmallest(least_m)

    # Call the function to draw the chart
    rx.draw_least_demand_chart(least_demand_sections, least_m, start_month)
else:
    print(f"Error: The input number should be between 1 and {max_sections}.")

After identifying the medications with the highest demand, there should also need to analyze those with the least demand over the same period for resource allocation. 

From the bar charts, there can be observe that categories like **Drainable Dribbling Appliance** and **Stoma care** have had almost no demand during this time. Excessive stock in these areas could lead to wastage of medical resources, as a large quantity of these medications may remain unused.

Healthcare departments can modify different inputs and use the provided code to generate charts, identifying the medications they consider to be in low demand.
 
Consequently, they can adjust the inventory of these categories, reallocating these resources to replenish the inventory of medications with higher demand, thereby enhancing the efficiency of medical resource utilization.

#### Prescription Resources Analysis

According to the previous section, the three most prevalent illnesses from 2018 to 2023 were **Hypertension**, **Depression**, and **Asthma**, while the three diseases that showed the highest rise in occurence were **Eating Disorders**, **Chronic Obstructive Pulmonary Disease (COPD)**, and **Stroke and Transient Ischemic Attack (TIA)**. In this section, we will have a look on the distributions of prescription sales of these diseases among all the Scotland NHS health boards in July of 2023. 

The description of certain drugs in the BNF reference can differ from the disease name. After doing some research, we found the following corresponding treatments that appear in the reference for the diseases:
- **Hypertension** ---- Hypertension and heart failure (BNF Section Code: 0205)
- **Depression** ---- Antidepressant drugs (BNF Section Code: 0403)
- **Asthma** ---- Antihistamines, hyposensitisation and allergic emergencies (BNF Section Code: 0304)
- **COPD** ---- Bronchodilators (BNF Section Code: 0301)
- **Stroke and TIA** ---- Antiplatelet drugs (BNF Section Code: 0209)

Note that **Eating Disorders** are not discussed here since it uses the same medications with **Depression** and what's more,  the primary treatment for eating disorders is usually a combination of psychological therapy and supervised weight gain, not medication.

Also note that there could be various treatments for one disease aiming at different effects. For instance, anti-dementia drugs are also used to treat **Depression** in some circumstances. Here we only consider the most popular prescription for simplexity.

##### Filter Out the Data of Targeted Prescription

Firstly, there is required to attain the data of targeted prescription for the disease that it is going to investigate. Here it is used customised function `flt_dict` in "prescription.py" to filter out the DataFrames for the prescription and store them in a dictionary.

In [ ]:
#Dictionary of BNF Section Code for the prescription of three prevalent diseases
bnf_code_curr = {
    'Hypertension': '0205',
    'Depression': '0403',
    'Asthma': '0304',
}

#Dictionary of the filtered DataFrames for the prescription of three prevalent diseases
prescription_data_dict_curr = rx.flt_dict(prescription_data, bnf_code_curr)

In [ ]:
#Dictionary of BNF Section Code for the prescription of two increasingly popular diseases
bnf_code_rise = {
    'COPD': '0301',
    'Stroke and TIA': '0209'
}

#Dictionary of the filtered DataFrames for the prescription of two increasingly popular diseases
prescription_data_dict_rise = rx.flt_dict(prescription_data, bnf_code_rise)

##### Choropleth Maps for Total Paid Quantity

In this part, there is going to use choropleth maps for the analysis since it could clearly show the distributions on the map. Procedures:
1. Load the shape file of Scotland maps, which is stored as GeoDataFrames. 
2. Use `groupsum` and `gdf_dict` functions to merge the shape file with the prescription dictionaries that previously set, resulting in dictionaries of GeoDataFrames for different prescriptions, which will be used for plotting.
3. Use `plot_single_choropleth` and `plot_multiple_choropleth` to make the choropleth maps for all, the top three prevalent, and the two increasingly popular diseases.

In [ ]:
#The GeoDataFrames for the prescription of all diseases (Paid Quantity)
prescription_gdf_all = rx.groupsum(prescription_data, shape_df, 'HBCode', 'PaidQuantity')

#Dictionary of the GeoDataFrames for the prescription of three prevalent diseases (Paid Quantity)
prescription_gdf_dict_curr = rx.gdf_dict(prescription_data, prescription_data_dict_curr, shape_df, 'HBCode', 'PaidQuantity')

#Dictionary of the GeoDataFrames for the prescription of two increasingly popular diseases (Paid Quantity)
prescription_gdf_dict_rise = rx.gdf_dict(prescription_data, prescription_data_dict_rise, shape_df, 'HBCode', 'PaidQuantity')

In [ ]:
#Choropleth Maps - all diseases (Paid Quantity)
print("Map for showing the distribution of prescription paid quantity for all diseases in July, 2023")
rx.plot_single_choropleth(prescription_gdf_all, "PaidQuantity", 'Purples', title='Prescriptions')

In [ ]:
#Choropleth Maps - top three prevalent diseases (Paid Quantity)
print("Map for showing the distribution of prescription paid quantity for the top three prevalent diseases in July, 2023")
rx.plot_multiple_choropleths(prescription_gdf_dict_curr, 'PaidQuantity', 'PuBu')

In [ ]:
#Choropleth Maps - two increasingly popular diseases (Paid Quantity)
print("Map for showing the distribution of prescription paid quantity for the two increasingly popular diseases in July, 2023")
rx.plot_multiple_choropleths(prescription_gdf_dict_rise, 'PaidQuantity', 'PuBu')

Here are the observations, for all the diseases:

- In July 2023 (lastest data), the prescriptions' paid quantity was high in the mid-southern part of Scotland, while that was relatively low in the northern islands.
- Total paid quantity was extremely high in *Greater Glasgow and Clyde* region, indicating high prevelance rate and high medication demand (parts of the reason could be higher population densities or greater access to medical facilities).

For the top three prevalent diseases (**Hypertension**, **Depression**, **Asthma**):

- The distribution patterns of the average prevalence rate over five years (plotted in the previous section) and prescription sales in July 2023 of these three diseases are not identical. This could suggest that medication prescription rates do not strictly follow prevalence patterns and might be influenced by other factors such as uneven investment of medical resourses. For instance, while the prevalence rates of these three diseases are very high in the north islands, the paid quantities of certain medication are very low, which could be a representation of insufficient medical resources in these areas. (<u>combine with the GP analysis</u>)

- Asthma prescriptions are relatively low in quantity compared to Hypertension and Depression, which could imply a lower demand for asthma-related medications or a lower rate of medication-controlled asthma.

For the two increasingly popular diseases (**COPD**, **Stroke and TIA**):

- The distribution of **COPD** prescription shares common pattern with most of the diseases, while there was low demand for **Stroke and TIA** treatments in this month.
- Among these five diseases, **COPD** has overall higher prescription quantities in all regions than other diseases, which might suggest **COPD** has a higher treatment rate with medication.

##### Choropleth Maps for Gross Ingredient Cost

This part analysis of prescriptions' gross ingredient cost aimed at finding regions where the paid quantity is high but the gross ingredient cost is relatively low, which may indicate cost-effective prescribing practices or the use of cheaper but effective medications.

In [ ]:
#The GeoDataFrames for the prescription of all diseases (Gross Ingredient Cost)
prescription_gdf_all2 = rx.groupsum(prescription_data, shape_df, 'HBCode', 'GrossIngredientCost')

#Dictionary of the GeoDataFrames for the prescription of three prevalent diseases (Gross Ingredient Cost)
prescription_gdf_dict_curr2 = rx.gdf_dict(prescription_data, prescription_data_dict_curr, shape_df, 'HBCode', 'GrossIngredientCost')

#Dictionary of the GeoDataFrames for the prescription of two increasingly popular diseases (Gross Ingredient Cost)
prescription_gdf_dict_rise2 = rx.gdf_dict(prescription_data, prescription_data_dict_rise, shape_df, 'HBCode', 'GrossIngredientCost')

In [ ]:
#Choropleth Maps - all diseases (Gross Ingredient Cost)
print("Map for showing the distribution of prescription Gross Ingredient Cost for all diseases in July, 2023")
rx.plot_single_choropleth(prescription_gdf_all2, "GrossIngredientCost", 'Greens', title='Prescriptions (Total Gross Ingredient Cost)')

In [ ]:
#Choropleth Maps - top three prevalent diseases (Gross Ingredient Cost)
print("Map for showing the distribution of prescription Gross Ingredient Cost for the top three prevalent diseases in July, 2023")
rx.plot_multiple_choropleths(prescription_gdf_dict_curr2, "GrossIngredientCost", 'YlGn')

In [ ]:
#Choropleth Maps - two increasingly popular diseases (Gross Ingredient Cost)
print("Map for showing the distribution of prescription Gross Ingredient Cost for the two increasingly popular diseases in July, 2023")
rx.plot_multiple_choropleths(prescription_gdf_dict_rise2, "GrossIngredientCost", 'YlGn')

There appears to be a correlation between the paid quantity of prescriptions and their gross ingredient cost among all the regions. Areas with a higher volume of prescriptions often have higher costs, which is expected as cost is typically a function of the quantity of items dispensed and their individual prices. 

Note that the exclusion of broken bulk in the cost measure might affect the accuracy of cost representations, particularly in areas where bulk prescriptions might be common.

## Conclusion

In conclusion, the analysis of the projected population and prevalent diseases in Scotland has provided valuable insights. The **population** and **diseased population** are both **growing**, with the number of infections expected to increase along with the population. Regression models and time series analysis are not feasible due to limited data and non-stationarity. Monte-Carlo Forecasting with Bayesian Theory is eventually proposed for future predictions. **Hypertension**, **depression**, and **asthma** are prevalent illnesses in Scotland, with age and gender being significant factors, such as **female** are more likely to get depression. The geographic distribution shows variations in disease prevalence, with higher rates in **northern regions**, especially in **Shetland**. These findings highlight the need for targeted healthcare interventions and greater attention to female mental health and resources allocation in Shetland.

Moreover, there is also reveals a **disparity** between the growing infectious population and the GP workforce in Scotland. The number of **male GPs exceeds that of females**, indicating a widening gender gap. **Greater Glasgow and Clyde** have the highest number of GPs, patients, and workforce. The low number of GPs in **Shetland** contrasts with the region's comparatively high prevalence rates for prevalent diseases. GP practices are **concentrated in the southern part of Scotland**, particularly in Edinburgh and Glasgow.

Last but not least, the analysis of prescription data and disease prevalence in Scotland provides valuable insights into medication demand and distribution across different **diseases** and **regions**. Medications such as **analgesics** and **drugs used in diabetes** have consistently had the **highest demand** over the past three years, while categories like **Drainable Dribbling Appliance** and **Stoma care** have shown **minimal demand**. The distribution of prescription sales for prevalent diseases also **does not strictly follow the prevalence patterns**, indicating potential disparities in medical resource allocation. Additionally, the analysis reveals variations in prescription quantities for different diseases, suggesting differences in treatment rates. Overall, these insights can guide healthcare departments in **improving medication supply** and **optimizing resource utilization**.


Base on all the above findings, there is going to give three suggestions:

1. **Improving Female Welfare in Working with NHS**: Female labor-force participation should be prioritized, since it continues to decline. Furthermore, according to the findings of this study, women are more prone to have a disease, particularly depression. Therefore, it is advised to give females greater metheal health assistance. In addition, there are welfare programs specifically designed for women that can be implemented. For example, Menstrual Leave helps female employees deal with the stress of approaching menstruation, and Childcare Support establishes or funds on-site childcare facilities or collaborates with nearby daycare centers to offer working mothers high-quality, reasonably priced childcare options.
2. **Enhancing Healthcare Resource Allocation System:** As the demand for medications varied across different diseases, it is recommended to allocate resources based on demand patterns. Medications with high demand, such as analgesics and diabetes drugs, should be adequately supplied, while low-demand medications should be carefully managed to avoid wastage.
3. **Addressing Regional Disparities:** Efforts should be undertaken to address regional inequities in healthcare resources and services. This includes ensuring a proper distribution of GP workforce, especially in places with high disease prevalence but low GP availability, such as Shetland. For example, workers who choose to work in locations with high prevalence of illnesses but poor GP availability may be paid more.
